# **ID3 Decision Tree**



# **Dependency Imports**

In [ ]:
import pandas as pd # data manipulation
import numpy as np  # mathematical calulation
import typing # helper for using tuple in python type hinting
from sklearn import datasets # importation of pre-exsiting datasets
from sklearn.model_selection import train_test_split # for splitting datasets into train/test splits
eps = np.finfo(float).eps # smallest possible positive number that can be represented. For avoiding div by 0 errors 
from pprint import pprint # for printing dict types in a more formatted way

# **ID3 Classes**
- A recursive and OOP designed implementation

In [ ]:
class SubTree:

  def __init__(self):
    self.tree = {} # the initial sub tree
  

  def createSubTree(self, 
                    featureName: str, 
                    trainData: pd.DataFrame, 
                    label: str, 
                    classList: list) -> typing.Tuple[dict, pd.DataFrame]:

    """Creates the a sub tree for the feature with the highest info gain.
    
    :param featureName: 
    :param trainData: passed in DataFrame containing training data
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    
    :return: dictionary of the subTree,
             updated dataset DataFrame
    """

    featureValueCount = trainData[featureName].value_counts(sort=False) # creates a dictionary containing the counts of unique features
    
    for featureValue, count in featureValueCount.iteritems(): # loop through the features and their counts
        featureValueData = trainData[trainData[featureName] == featureValue] 
        
        pureClass = False # flag for whether the feature value has only one target label type. i.e is 'pure'
        for c in classList: 
            classCount = featureValueData[featureValueData[label] == c].shape[0] 

            if classCount == count: # if the counts match the feature is 'pure'
                self.tree[featureValue] = c # add to the sub tree
                trainData = trainData[trainData[featureName] != featureValue] # remove the feature value rows from the dataset
                pureClass = True
        if not pureClass: 
            self.tree[featureValue] = "?" # mark with '?' as the branch can be extended further
            
    return self.tree, trainData

In [ ]:
class Tree:

  def __init__(self, trainData: pd.DataFrame, label: str): 
    classList = trainData[label].unique()

    self.tree = {} # initialize the main tree dictionary
    self.createTree(self.tree, None, trainData, label, classList) # create and train the tree


  def createTree(self, 
                 root: object, 
                 prevFeatureValue: type, 
                 trainData: pd.DataFrame, 
                 label: str, 
                 classList: list):
    
    """Creates and trains the main tree.
    
    :param root: current node of the tree that is being worked on
    :param prevFeatureValue: the value of the previous feature
    :param trainData: passed in DataFrame containing training data
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    """
    
    if trainData.shape[0] != 0: # if the updated dataset is not empty
      print("GENERATING TREE SEGMENT")
      maxInfoFeature = self.highestInfoFeature(trainData, label, classList) # find the feature with the highest info
      subTree = SubTree() 
      subTreeStructure, newTrainData = subTree.createSubTree(maxInfoFeature, trainData, label, classList) # create a sub tree
      nextRoot = None
      
      if prevFeatureValue != None: # add values to end node
          root[prevFeatureValue] = dict()
          root[prevFeatureValue][maxInfoFeature] = subTreeStructure
          nextRoot = root[prevFeatureValue][maxInfoFeature]
      else: # add values to tree root
          root[maxInfoFeature] = subTreeStructure
          nextRoot = root[maxInfoFeature]
      
      for node, branch in list(nextRoot.items()): # iterating through the tree node
          if branch == "?": # if the branch can still be expanded
              featureData = trainData[trainData[maxInfoFeature] == node] # updated dataset
              self.createTree(nextRoot, node, featureData, label, classList) # recursive call with the updated dataset


  def highestInfoFeature(self, 
                         trainData: pd.DataFrame, 
                         label: str, 
                         classList: list) -> str:

    """Calculates and returns the feature within the DataFrame that has the highest information gain.
    
    :param trainData: passed in DataFrame containing training data
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    
    :return: str value of the feature with the highest info gain
    """

    featureList = trainData.columns.drop(label) # drop the target label and its associated column from the DataFrame
                                            
    maxGain = -1
    maxFeature = None
    
    for index, feature in enumerate(featureList): # loop through all features 
        print("Calculating feature {featureNumber} out of {totalFeatures}...".format(featureNumber=index+1, totalFeatures=len(featureList)))
        featureInfoGain = self.infoGain(feature, trainData, label, classList) # calculate the info gain for the feature
        if maxGain < featureInfoGain: # changing the maxGain and maxFeature if the feature has the current highest info gain
            maxGain = featureInfoGain
            maxFeature = feature
            
    return maxFeature


  def infoGain(self, 
               featureName: str, 
               trainData: pd.DataFrame, 
               label: str, 
               classList: list) -> float: 

    """Calculates the total entropy of the passed in DataFrame.
    
    :param featureName: The passed in name of the feature to calculate the info gain for
    :param trainData: passed in DataFrame containing training data
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    
    :return: float value for the features calculated info gain
    """

    featureValueList = trainData[featureName].unique() # unique values for the specific feature
    rowTotal = trainData.shape[0] # total amount of rows in the Dataframe
    featureInfo = 0.0
    
    for featureValue in featureValueList: # rows containing the featureName
        featureValueData = trainData[trainData[featureName] == featureValue] 
        featureValueCount = featureValueData.shape[0]
        featureValueEntropy = self.calcEntropy(featureValueData, label, classList) # entropy calculation
        featureValueProbability = featureValueCount/rowTotal # calculate the probability of the feature value
        featureInfo += featureValueProbability * featureValueEntropy 
        
    return self.totalEntropy(trainData, label, classList) - featureInfo # subtract the feature value information from the total entropy to get the info gain
    

  def totalEntropy(self,
                   trainData: pd.DataFrame, 
                   label: str, 
                   classList: list) -> float:

    """Calculates the total entropy of the passed in DataFrame.
    
    :param trainData: the passed in DataFrame from the infoGain function
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    
    :return: float value for the DataFrame's total entropy
    """

    rowTotal = trainData.shape[0] # dataset size
    entropyTotal = 0 
    
    for c in classList: # loop through all the classes in classList
        totalClassCount = trainData[trainData[label] == c].shape[0] # which class it is within classList
        totalClassEntr = - (totalClassCount/rowTotal)*np.log2(totalClassCount/rowTotal+eps) # calculate the entropy of the class
        entropyTotal += totalClassEntr # add that classes entropy to the total
    
    return entropyTotal


  def calcEntropy(self, 
                  featureData: pd.DataFrame, 
                  label: str, 
                  classList: list) -> float:

    """Calculates the entropy of a filtered DataFrame that contains rows with only one specific feature value.
    
    :param featureData: the passed in filtered DataFrame 
    :param label: target label associated with the DataFrame
    :param classList: list of unique classes for the label
    
    :return: float value for the filtered DataFrame's entropy
    """

    classCount = featureData.shape[0]
    entropy = 0
    
    for c in classList: # loop through all the classes in classList
        labelClassCount = featureData[featureData[label] == c].shape[0] # which class it is within classList
        entropyClass = 0
        if labelClassCount != 0:
            probabilityClass = labelClassCount/classCount # class probability
            entropyClass = - probabilityClass * np.log2(probabilityClass) # calculate the entropy of the class
        entropy += entropyClass # add that classes entropy to the total
    return entropy

In [ ]:
class Id3:
  def __init__(self, 
               trainData: pd.DataFrame, 
               label: str): 

    self.tree = Tree(trainData, label) # initialize the Tree class with the training data and target label name
    
  def printTree(self):
    """Prints the whole tree dictionary in a clean format."""
    pprint(self.tree.tree) 


  def predict(self, 
              tree: dict, 
              instance: type) -> str:

    """Predicts a row of test data using the fully trained tree dictionary.
    
    :param tree: the trained tree dictionary 
    :param instance: a row of a DataFrame
    
    :return: predicted target string of the row
    """

    if not isinstance(tree, dict): # if leaf node
        return tree 
    else:
        rootNode = next(iter(tree)) # first feature in the dict
        featureValue = instance[rootNode] 
        if featureValue in tree [rootNode]: # check feature value
            return self.predict(tree [rootNode][featureValue], instance) # recursively go to the next feature in the dict
        else:
            return None


  def evaluate(self, 
               testData: pd.DataFrame, 
               label: str) -> float: 

    """Loops through the test data to predict each row and calculate the models accuracy.
    
    :param testData: the test data DataFrame
    :param label: the target label to be predicted
    
    :return: the accuracy score of the model on the test data
    """

    correctPred = 0
    wrongPred = 0
    for index, row in testData.iterrows(): # each dataset row
        result = self.predict(self.tree.tree, testData.iloc[index]) # run predict on the rows data
        if result == testData[label].iloc[index]: # if the predicted value = the true value
            correctPred += 1
        else:
            wrongPred += 1 
    accuracy = correctPred / (correctPred + wrongPred) # calculate accuracy
    return accuracy

# **Using Iris Dataset**

### **Dataset Preprocessing**

In [ ]:
# Load iris dataset from sklearn.dataset
irisDataset = datasets.load_iris()

In [ ]:
# Display dataset feature and target names
print(irisDataset.feature_names)
print(irisDataset.target_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [ ]:
# Convert the dataset into a pandas DataFrame
df = pd.DataFrame(data=irisDataset.data, columns=irisDataset.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
# Loop through the target data (currently the the format of target_names array indices) 
# and create a new array with the string equivalents
newTargetData = []

for targetValue in irisDataset.target:
  newTargetData.append(irisDataset.target_names[targetValue])
print(newTargetData)

['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicolor', 'versicol

In [ ]:
# Append the new target data array to the main data DataFrame
df.insert(loc=4, column='Target', value=newTargetData)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [ ]:
# Split and shuffle the dataset into training and testing sets at a 80:20 split respectively
irisTrain, irisTest = train_test_split(df, test_size=0.20, random_state=42, shuffle=True)

In [ ]:
# Reset the indices as the data shuffling caused them to be random and unordered
irisTrain.reset_index(drop=True, inplace=True)
irisTest.reset_index(drop=True, inplace=True)
irisTest.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,6.1,2.8,4.7,1.2,versicolor
1,5.7,3.8,1.7,0.3,setosa
2,7.7,2.6,6.9,2.3,virginica
3,6.0,2.9,4.5,1.5,versicolor
4,6.8,2.8,4.8,1.4,versicolor


### **Training Tree and Making Predictions**

In [ ]:
# Create and train tree on the training data (0-1sec run time avg)
irisId3 = Id3(irisTrain, 'Target')

GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...
GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...
GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...
GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...
GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...
GENERATING TREE SEGMENT
Calculating feature 1 out of 4...
Calculating feature 2 out of 4...
Calculating feature 3 out of 4...
Calculating feature 4 out of 4...


In [ ]:
# Display the trained tree
irisId3.printTree()

{'petal length (cm)': {1.0: 'setosa',
                       1.1: 'setosa',
                       1.2: 'setosa',
                       1.3: 'setosa',
                       1.4: 'setosa',
                       1.5: 'setosa',
                       1.6: 'setosa',
                       1.7: 'setosa',
                       1.9: 'setosa',
                       3.0: 'versicolor',
                       3.3: 'versicolor',
                       3.5: 'versicolor',
                       3.7: 'versicolor',
                       3.8: 'versicolor',
                       3.9: 'versicolor',
                       4.0: 'versicolor',
                       4.1: 'versicolor',
                       4.2: 'versicolor',
                       4.3: 'versicolor',
                       4.4: 'versicolor',
                       4.5: {'sepal length (cm)': {4.9: 'virginica',
                                                   5.4: 'versicolor',
                                                   5.6: '

In [ ]:
# Evaluate the trained id3 decision tree accuracy on the test data
irisId3.evaluate(irisTest, 'Target')

0.7

# **Using Kddcup99 Dataset**

### **Dataset Preprocessing**

In [ ]:
# Load iris dataset from sklearn.dataset
kddDataset = datasets.fetch_kddcup99()

In [ ]:
# Display dataset feature
print(kddDataset.feature_names)

['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate']


In [ ]:
# Diplay dataset target names using the set function on target array (dataset.target_names did not contain the names in this case)
print(set(kddDataset.target))

{b'ipsweep.', b'normal.', b'pod.', b'neptune.', b'nmap.', b'land.', b'back.', b'teardrop.', b'spy.', b'portsweep.', b'perl.', b'loadmodule.', b'phf.', b'warezclient.', b'multihop.', b'guess_passwd.', b'imap.', b'smurf.', b'ftp_write.', b'satan.', b'rootkit.', b'buffer_overflow.', b'warezmaster.'}


In [ ]:
# Convert the dataset into a pandas DataFrame
df2 = pd.DataFrame(data=kddDataset.data, columns=kddDataset.feature_names)
df2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Append the new target data array to the main data DataFrame
df2.insert(loc=41, column='Target', value=kddDataset.target)
df2.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Target
0,0,b'tcp',b'http',b'SF',181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,b'normal.'
1,0,b'tcp',b'http',b'SF',239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,b'normal.'
2,0,b'tcp',b'http',b'SF',235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
3,0,b'tcp',b'http',b'SF',219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,b'normal.'
4,0,b'tcp',b'http',b'SF',217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,b'normal.'


In [ ]:
# Split the dataset by 50% due to too many samples as well as shuffle it
train, test = train_test_split(df2, test_size=0.5, random_state=42, shuffle=True)

In [ ]:
# Split and shuffle one of the previously split datasets into training and testing sets at a 80:20 split respectively
traint, testt = train_test_split(test, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
# Reset the indices as the data shuffling caused them to be random and unordered
traint.reset_index(drop=True, inplace=True)
testt.reset_index(drop=True, inplace=True)
testt.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Target
0,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,b'smurf.'
1,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,b'smurf.'
2,0,b'icmp',b'ecr_i',b'SF',1032,0,0,0,0,0,...,255,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,b'smurf.'
3,0,b'tcp',b'private',b'S0',0,0,0,0,0,0,...,20,0.08,0.06,0.0,0.0,1.0,1.0,0.0,0.0,b'neptune.'
4,0,b'tcp',b'private',b'S0',0,0,0,0,0,0,...,4,0.02,0.08,0.0,0.0,1.0,1.0,0.0,0.0,b'neptune.'


In [ ]:
# Create and train tree on the training data (25-30min runtime avg)
KddId3 = Id3(traint, 'Target')

Streaming output truncated to the last 5000 lines.
Calculating feature 40 out of 41...
Calculating feature 41 out of 41...
GENERATING TREE SEGMENT
Calculating feature 1 out of 41...
Calculating feature 2 out of 41...
Calculating feature 3 out of 41...
Calculating feature 4 out of 41...
Calculating feature 5 out of 41...
Calculating feature 6 out of 41...
Calculating feature 7 out of 41...
Calculating feature 8 out of 41...
Calculating feature 9 out of 41...
Calculating feature 10 out of 41...
Calculating feature 11 out of 41...
Calculating feature 12 out of 41...
Calculating feature 13 out of 41...
Calculating feature 14 out of 41...
Calculating feature 15 out of 41...
Calculating feature 16 out of 41...
Calculating feature 17 out of 41...
Calculating feature 18 out of 41...
Calculating feature 19 out of 41...
Calculating feature 20 out of 41...
Calculating feature 21 out of 41...
Calculating feature 22 out of 41...
Calculating feature 23 out of 41...
Calculating feature 24 out of 41..

In [ ]:
# Display the trained tree
KddId3.printTree()

{'src_bytes': {0: {'dst_host_diff_srv_rate': {0.0: {'service': {b'finger': b'land'
                                                                           b'.',
                                                                b'ftp_data': {'dst_bytes': {0: b'ipsw'
                                                                                               b'eep.',
                                                                                            848: b'ware'
                                                                                                 b'zmas'
                                                                                                 b'ter.',
                                                                                            5696: b'norm'
                                                                                                  b'al.',
                                                                                            5928: b'b

In [ ]:
# Evaluate the trained id3 decision tree accuracy on the test data
KddId3.evaluate(testt, 'Target')

0.9934416938242617